## Introduktion

Denne lektion vil dække:
- Hvad funktionkald er, og dets anvendelsestilfælde
- Hvordan man opretter et funktionkald ved hjælp af OpenAI
- Hvordan man integrerer et funktionkald i en applikation

## Læringsmål

Efter at have gennemført denne lektion vil du vide, hvordan man gør, og forstå:

- Formålet med at bruge funktionkald
- Opsæt funktionkald ved hjælp af OpenAI-tjenesten
- Design effektive funktionkald til din applikations anvendelsestilfælde


## Forståelse af Funktionskald

Til denne lektion vil vi bygge en funktion til vores uddannelsesstartup, der giver brugerne mulighed for at bruge en chatbot til at finde tekniske kurser. Vi vil anbefale kurser, der passer til deres færdighedsniveau, nuværende rolle og interesse for teknologi.

For at fuldføre dette vil vi bruge en kombination af:
 - `OpenAI` til at skabe en chatoplevelse for brugeren
 - `Microsoft Learn Catalog API` til at hjælpe brugerne med at finde kurser baseret på brugerens forespørgsel
 - `Function Calling` til at tage brugerens forespørgsel og sende den til en funktion for at lave API-forespørgslen.

For at komme i gang, lad os se på, hvorfor vi overhovedet vil bruge funktionskald:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # få et nyt svar fra GPT, hvor det kan se funktionssvaret


print(second_response.choices[0].message)


### Hvorfor Funktionskald

Hvis du har gennemført en anden lektion i dette kursus, forstår du sandsynligvis kraften ved at bruge Store Sprogmodeller (LLMs). Forhåbentlig kan du også se nogle af deres begrænsninger.

Funktionskald er en funktion i OpenAI Service designet til at løse følgende udfordringer:

Uensartet svarformat:
- Før funktionskald var svar fra en stor sprogmodel ustrukturerede og inkonsistente. Udviklere måtte skrive kompleks valideringskode for at håndtere hver variation i outputtet.

Begrænset integration med eksterne data:
- Før denne funktion var det svært at inkorporere data fra andre dele af en applikation i en chat-kontekst.

Ved at standardisere svarformater og muliggøre problemfri integration med eksterne data forenkler funktionskald udviklingen og reducerer behovet for yderligere valideringslogik.

Brugere kunne ikke få svar som "Hvad er vejret lige nu i Stockholm?". Dette skyldes, at modellerne var begrænset til det tidspunkt, hvor dataene blev trænet.

Lad os se på eksemplet nedenfor, der illustrerer dette problem:

Lad os sige, at vi vil oprette en database med elevdata, så vi kan foreslå det rigtige kursus til dem. Nedenfor har vi to beskrivelser af elever, der er meget ens i de data, de indeholder.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Vi vil sende dette til en LLM for at analysere dataene. Dette kan senere bruges i vores applikation til at sende det til en API eller gemme det i en database.

Lad os oprette to identiske prompts, hvor vi instruerer LLM om, hvilken information vi er interesserede i:


Vi vil sende dette til en LLM for at analysere de dele, der er vigtige for vores produkt. Så vi kan oprette to identiske prompts til at instruere LLM'en:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Efter at have oprettet disse to prompts, sender vi dem til LLM ved at bruge `openai.ChatCompletion`. Vi gemmer prompten i variablen `messages` og tildeler rollen `user`. Dette er for at efterligne en besked fra en bruger, der skrives til en chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nu kan vi sende begge forespørgsler til LLM og undersøge det svar, vi modtager.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Selvom promptene er de samme, og beskrivelserne ligner hinanden, kan vi få forskellige formater af `Grades`-egenskaben.

Hvis du kører ovenstående celle flere gange, kan formatet være `3.7` eller `3.7 GPA`.

Dette skyldes, at LLM tager ustrukturerede data i form af den skrevne prompt og også returnerer ustrukturerede data. Vi har brug for at have et struktureret format, så vi ved, hvad vi kan forvente, når vi gemmer eller bruger disse data.

Ved at bruge funktionel kald kan vi sikre, at vi modtager strukturerede data tilbage. Når vi bruger funktionel kald, kalder eller kører LLM faktisk ikke nogen funktioner. I stedet opretter vi en struktur, som LLM skal følge for sine svar. Vi bruger derefter disse strukturerede svar til at vide, hvilken funktion vi skal køre i vores applikationer.


![Funktionskald Flow Diagram](../../../../translated_images/da/Function-Flow.083875364af4f4bb.webp)


Vi kan derefter tage det, der returneres fra funktionen, og sende det tilbage til LLM'en. LLM'en vil så svare ved hjælp af naturligt sprog for at besvare brugerens forespørgsel.


### Brugstilfælde for brug af funktionskald

**Kald af eksterne værktøjer**  
Chatbots er gode til at give svar på spørgsmål fra brugere. Ved at bruge funktionskald kan chatbots bruge beskeder fra brugere til at udføre bestemte opgaver. For eksempel kan en studerende bede chatbotten om at "Sende en e-mail til min underviser og sige, at jeg har brug for mere hjælp med dette emne". Dette kan lave et funktionskald til `send_email(to: string, body: string)`

**Opret API- eller databaseforespørgsler**  
Brugere kan finde information ved hjælp af naturligt sprog, som bliver konverteret til en formateret forespørgsel eller API-anmodning. Et eksempel på dette kunne være en lærer, der spørger "Hvem er de studerende, der har fuldført den sidste opgave", hvilket kunne kalde en funktion med navnet `get_completed(student_name: string, assignment: int, current_status: string)`

**Oprettelse af strukturerede data**  
Brugere kan tage et tekstafsnit eller CSV og bruge LLM til at udtrække vigtig information fra det. For eksempel kan en studerende konvertere en Wikipedia-artikel om fredsaftaler til at lave AI-flashcards. Dette kan gøres ved at bruge en funktion kaldet `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Oprettelse af dit første funktionskald

Processen med at oprette et funktionskald inkluderer 3 hovedtrin:
1. Kalde Chat Completions API med en liste over dine funktioner og en brugermeddelelse
2. Læse modellens svar for at udføre en handling, dvs. køre en funktion eller API-kald
3. Foretage et nyt kald til Chat Completions API med svaret fra din funktion for at bruge den information til at skabe et svar til brugeren.


![Flow af et funktionskald](../../../../translated_images/da/LLM-Flow.3285ed8caf4796d7.webp)


### Elementer i et funktionskald

#### Brugerinput

Det første trin er at oprette en brugermeddelelse. Dette kan tildeles dynamisk ved at tage værdien af en tekstinput, eller du kan tildele en værdi her. Hvis det er første gang, du arbejder med Chat Completions API, skal vi definere `role` og `content` i meddelelsen.

`role` kan enten være `system` (opretter regler), `assistant` (modellen) eller `user` (slutbrugeren). Til funktionskald vil vi tildele denne som `user` og et eksempelspørgsmål.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Oprettelse af funktioner.

Næste trin er at definere en funktion og parametrene for den funktion. Vi vil bruge kun én funktion her kaldet `search_courses`, men du kan oprette flere funktioner.

**Vigtigt** : Funktioner er inkluderet i systembeskeden til LLM og vil tælle med i det antal tilgængelige tokens, du har til rådighed.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitioner** 

Funktionsdefinitionsstrukturen har flere niveauer, hver med sine egne egenskaber. Her er en oversigt over den indlejrede struktur:

**Topniveau Funktions Egenskaber:**

`name` - Navnet på funktionen, som vi ønsker skal kaldes. 

`description` - Dette er beskrivelsen af, hvordan funktionen fungerer. Her er det vigtigt at være specifik og klar 

`parameters` - En liste over værdier og format, som du ønsker, at modellen skal producere i sit svar 

**Egenskaber for Parameterobjektet:**

`type` - Datatypen for parameterobjektet (normalt "object")

`properties` - Liste over de specifikke værdier, som modellen vil bruge til sit svar 

**Egenskaber for Individuelle Parametre:**

`name` - Implicit defineret af egenskabsnøglen (f.eks. "role", "product", "level")

`type` - Datatypen for denne specifikke parameter (f.eks. "string", "number", "boolean") 

`description` - Beskrivelse af den specifikke parameter 

**Valgfrie Egenskaber:**

`required` - Et array, der angiver hvilke parametre, der er nødvendige for, at funktionskaldet kan gennemføres 


### Foretage funktionskaldet  
Efter at have defineret en funktion, skal vi nu inkludere den i kaldet til Chat Completion API'en. Det gør vi ved at tilføje `functions` til forespørgslen. I dette tilfælde `functions=functions`.  

Der er også en mulighed for at sætte `function_call` til `auto`. Det betyder, at vi lader LLM'en beslutte, hvilken funktion der skal kaldes baseret på brugermeddelelsen i stedet for at tildele det selv.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Lad os nu se på svaret og se, hvordan det er formateret:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Du kan se, at navnet på funktionen bliver kaldt, og ud fra brugerens besked var LLM i stand til at finde dataene, så de passer til funktionens argumenter.


## 3. Integrering af funktionskald i en applikation. 


Efter vi har testet det formaterede svar fra LLM, kan vi nu integrere dette i en applikation. 

### Styring af flowet 

For at integrere dette i vores applikation, lad os tage følgende skridt: 

Først laver vi kaldet til OpenAI-tjenesterne og gemmer beskeden i en variabel kaldet `response_message`. 


In [ ]:
response_message = response.choices[0].message

Nu vil vi definere funktionen, der vil kalde Microsoft Learn API'en for at få en liste over kurser:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Som en bedste praksis vil vi derefter se, om modellen ønsker at kalde en funktion. Derefter vil vi oprette en af de tilgængelige funktioner og matche den med den funktion, der bliver kaldt.  
Vi vil derefter tage argumenterne til funktionen og kortlægge dem til argumenter fra LLM.

Til sidst vil vi tilføje funktionskaldsbeskeden og de værdier, der blev returneret af `search_courses`-beskeden. Dette giver LLM alle de oplysninger, den har brug for,  
for at svare brugeren ved hjælp af naturligt sprog.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Nu sender vi den opdaterede besked til LLM, så vi kan modtage et svar i naturligt sprog i stedet for et API JSON-formateret svar.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kodeudfordring

Godt arbejde! For at fortsætte din læring af OpenAI Function Calling kan du bygge: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Flere parametre til funktionen, som kan hjælpe lærende med at finde flere kurser. Du kan finde de tilgængelige API-parametre her:  
 - Opret et andet funktionskald, der tager flere oplysninger fra den lærende, såsom deres modersmål  
 - Opret fejlhåndtering, når funktionskaldet og/eller API-kaldet ikke returnerer nogen egnede kurser  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, bedes du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det oprindelige dokument på dets modersmål bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
